In [1]:
import subprocess
import sys
import os
import shutil
from pathlib import Path

# Check if UV is installed
def check_uv_installed():
    try:
        result = subprocess.run(['uv', '--version'], capture_output=True, text=True)
        return result.returncode == 0
    except FileNotFoundError:
        return False

# Install UV if not present
if not check_uv_installed():
    print("Installing UV package manager...")
    subprocess.run(['curl', '-LsSf', 'https://astral.sh/uv/install.sh', '|', 'sh'], shell=True)
    # Add UV to PATH for current session
    home_dir = Path.home()
    uv_path = home_dir / '.local' / 'bin'
    if str(uv_path) not in os.environ['PATH']:
        os.environ['PATH'] = f"{uv_path}:{os.environ['PATH']}"
else:
    print("UV is already installed")
    
# Verify UV installation
try:
    result = subprocess.run(['uv', '--version'], capture_output=True, text=True)
    print(f"UV version: {result.stdout.strip()}")
except Exception as e:
    print(f"Error checking UV: {e}")
    print("Please install UV manually: curl -LsSf https://astral.sh/uv/install.sh | sh")


UV is already installed
UV version: uv 0.7.4


In [2]:
# Clone TRELLIS repository if not already present
if not os.path.exists('TRELLIS'):
    print("Cloning TRELLIS repository...")
    subprocess.run(['git', 'clone', '--recurse-submodules', 'https://github.com/microsoft/TRELLIS.git'])
    os.chdir('TRELLIS')
else:
    print("TRELLIS repository already exists")
    os.chdir('TRELLIS')

print(f"Working directory: {os.getcwd()}")


Cloning TRELLIS repository...


Cloning into 'TRELLIS'...
Submodule 'trellis/representations/mesh/flexicubes' (https://github.com/MaxtirError/FlexiCubes.git) registered for path 'trellis/representations/mesh/flexicubes'
Cloning into '/root/TRELLIS/TRELLIS/trellis/representations/mesh/flexicubes'...


Submodule path 'trellis/representations/mesh/flexicubes': checked out '815e075a2a400d06c48d94c347674344ed6ae5c5'
Working directory: /root/TRELLIS/TRELLIS


In [ ]:
# Install PyTorch with CUDA support
print("Installing PyTorch with CUDA support...")
subprocess.run(['uv', 'pip', 'install', 'torch==2.4.0', 'torchvision==0.19.0', 
               '--index-url', 'https://download.pytorch.org/whl/cu118'])

# Install basic dependencies
print("Installing basic dependencies...")
basic_deps = [
    'pillow', 'imageio', 'imageio-ffmpeg', 'tqdm', 'easydict', 
    'opencv-python-headless', 'scipy', 'ninja', 'rembg', 'onnxruntime',
    'trimesh', 'open3d', 'xatlas', 'pyvista', 'pymeshfix', 'igraph', 'transformers'
]

cmd = ['uv', 'pip', 'install'] + basic_deps
result = subprocess.run(cmd, capture_output=True, text=True)
if result.returncode != 0:
    print(f"Error installing basic dependencies: {result.stderr}")
else:
    print("Basic dependencies installed successfully")

# Install utils3d
print("Installing utils3d...")
cmd = ['uv', 'pip', 'install', 'git+https://github.com/EasternJournalist/utils3d.git@9a4eb15e4021b67b12c460c7057d642626897ec8']
result = subprocess.run(cmd, capture_output=True, text=True)
if result.returncode != 0:
    print(f"Error installing utils3d: {result.stderr}")
else:
    print("utils3d installed successfully")

# Install Gradio for the web interface
print("Installing Gradio dependencies...")
subprocess.run(['uv', 'pip', 'install', 'pydantic<2.10,>=2.0'])
subprocess.run(['uv', 'pip', 'install', 'gradio==4.44.1', 'gradio_litmodel3d==0.0.1'])

print("✅ Dependencies installation completed!")


In [ ]:
# Set environment variables for optimal performance
os.environ['SPCONV_ALGO'] = 'native'  # Use native algorithm for single runs
os.environ['ATTN_BACKEND'] = 'xformers'  # Use xformers for attention

# Import required modules
import torch
import numpy as np
import imageio
from PIL import Image
from trellis.pipelines import TrellisImageTo3DPipeline
from trellis.utils import render_utils, postprocessing_utils

print("Environment variables set:")
print(f"SPCONV_ALGO: {os.environ.get('SPCONV_ALGO')}")
print(f"ATTN_BACKEND: {os.environ.get('ATTN_BACKEND')}")

# Load the pipeline
print("Loading TRELLIS pipeline... (This may take a few minutes)")
pipeline = TrellisImageTo3DPipeline.from_pretrained("microsoft/TRELLIS-image-large")
pipeline.cuda()
print("✅ Pipeline loaded successfully!")

# Check GPU memory usage
if torch.cuda.is_available():
    memory_allocated = torch.cuda.memory_allocated() / 1024**3
    memory_reserved = torch.cuda.memory_reserved() / 1024**3
    print(f"GPU Memory - Allocated: {memory_allocated:.2f}GB, Reserved: {memory_reserved:.2f}GB")


In [ ]:
import gradio as gr
from gradio_litmodel3d import LitModel3D
import tempfile
from typing import Tuple, List
from easydict import EasyDict as edict
from trellis.representations import Gaussian, MeshExtractResult

MAX_SEED = np.iinfo(np.int32).max

def preprocess_image(image: Image.Image) -> Image.Image:
    """Preprocess the input image."""
    return pipeline.preprocess_image(image)

def pack_state(gs: Gaussian, mesh: MeshExtractResult) -> dict:
    return {
        'gaussian': {
            **gs.init_params,
            '_xyz': gs._xyz.cpu().numpy(),
            '_features_dc': gs._features_dc.cpu().numpy(),
            '_scaling': gs._scaling.cpu().numpy(),
            '_rotation': gs._rotation.cpu().numpy(),
            '_opacity': gs._opacity.cpu().numpy(),
        },
        'mesh': {
            'vertices': mesh.vertices.cpu().numpy(),
            'faces': mesh.faces.cpu().numpy(),
        },
    }

def unpack_state(state: dict) -> Tuple[Gaussian, edict]:
    gs = Gaussian(
        aabb=state['gaussian']['aabb'],
        sh_degree=state['gaussian']['sh_degree'],
        mininum_kernel_size=state['gaussian']['mininum_kernel_size'],
        scaling_bias=state['gaussian']['scaling_bias'],
        opacity_bias=state['gaussian']['opacity_bias'],
        scaling_activation=state['gaussian']['scaling_activation'],
    )
    gs._xyz = torch.tensor(state['gaussian']['_xyz'], device='cuda')
    gs._features_dc = torch.tensor(state['gaussian']['_features_dc'], device='cuda')
    gs._scaling = torch.tensor(state['gaussian']['_scaling'], device='cuda')
    gs._rotation = torch.tensor(state['gaussian']['_rotation'], device='cuda')
    gs._opacity = torch.tensor(state['gaussian']['_opacity'], device='cuda')
    
    mesh = edict(
        vertices=torch.tensor(state['mesh']['vertices'], device='cuda'),
        faces=torch.tensor(state['mesh']['faces'], device='cuda'),
    )
    
    return gs, mesh

def get_seed(randomize_seed: bool, seed: int) -> int:
    """Get the random seed."""
    return np.random.randint(0, MAX_SEED) if randomize_seed else seed

def image_to_3d(
    image: Image.Image,
    seed: int,
    ss_guidance_strength: float,
    ss_sampling_steps: int,
    slat_guidance_strength: float,
    slat_sampling_steps: int,
) -> Tuple[dict, str]:
    """Convert an image to a 3D model."""
    
    outputs = pipeline.run(
        image,
        seed=seed,
        formats=["gaussian", "mesh"],
        preprocess_image=False,
        sparse_structure_sampler_params={
            "steps": ss_sampling_steps,
            "cfg_strength": ss_guidance_strength,
        },
        slat_sampler_params={
            "steps": slat_sampling_steps,
            "cfg_strength": slat_guidance_strength,
        },
    )
    
    # Render video
    video = render_utils.render_video(outputs['gaussian'][0], num_frames=120)['color']
    video_geo = render_utils.render_video(outputs['mesh'][0], num_frames=120)['normal']
    video = [np.concatenate([video[i], video_geo[i]], axis=1) for i in range(len(video))]
    
    # Save video to temporary file
    with tempfile.NamedTemporaryFile(suffix='.mp4', delete=False) as tmp_file:
        video_path = tmp_file.name
        imageio.mimsave(video_path, video, fps=15)
    
    state = pack_state(outputs['gaussian'][0], outputs['mesh'][0])
    torch.cuda.empty_cache()
    
    return state, video_path

def extract_glb(
    state: dict,
    mesh_simplify: float,
    texture_size: int,
) -> Tuple[str, str]:
    """Extract a GLB file from the 3D model."""
    gs, mesh = unpack_state(state)
    glb = postprocessing_utils.to_glb(gs, mesh, simplify=mesh_simplify, texture_size=texture_size, verbose=False)
    
    with tempfile.NamedTemporaryFile(suffix='.glb', delete=False) as tmp_file:
        glb_path = tmp_file.name
        glb.export(glb_path)
    
    torch.cuda.empty_cache()
    return glb_path, glb_path

def extract_gaussian(state: dict) -> Tuple[str, str]:
    """Extract a Gaussian file from the 3D model."""
    gs, _ = unpack_state(state)
    
    with tempfile.NamedTemporaryFile(suffix='.ply', delete=False) as tmp_file:
        gaussian_path = tmp_file.name
        gs.save_ply(gaussian_path)
    
    torch.cuda.empty_cache()
    return gaussian_path, gaussian_path

print("✅ Gradio interface functions defined successfully!")


In [ ]:
# Create Gradio interface
with gr.Blocks(title="TRELLIS: Image to 3D Asset") as demo:
    gr.Markdown("""
    ## Image to 3D Asset with [TRELLIS](https://trellis3d.github.io/)
    * Upload an image and click "Generate" to create a 3D asset.
    * If you find the generated 3D asset satisfactory, click "Extract GLB" to download it.
    """)
    
    with gr.Row():
        with gr.Column():
            image_prompt = gr.Image(label="Image Prompt", format="png", image_mode="RGBA", type="pil", height=300)
            
            with gr.Accordion(label="Generation Settings", open=False):
                seed = gr.Slider(0, MAX_SEED, label="Seed", value=0, step=1)
                randomize_seed = gr.Checkbox(label="Randomize Seed", value=True)
                gr.Markdown("Stage 1: Sparse Structure Generation")
                with gr.Row():
                    ss_guidance_strength = gr.Slider(0.0, 10.0, label="Guidance Strength", value=7.5, step=0.1)
                    ss_sampling_steps = gr.Slider(1, 50, label="Sampling Steps", value=12, step=1)
                gr.Markdown("Stage 2: Structured Latent Generation")
                with gr.Row():
                    slat_guidance_strength = gr.Slider(0.0, 10.0, label="Guidance Strength", value=3.0, step=0.1)
                    slat_sampling_steps = gr.Slider(1, 50, label="Sampling Steps", value=12, step=1)

            generate_btn = gr.Button("Generate", variant="primary")
            
            with gr.Accordion(label="GLB Extraction Settings", open=False):
                mesh_simplify = gr.Slider(0.9, 0.98, label="Simplify", value=0.95, step=0.01)
                texture_size = gr.Slider(512, 2048, label="Texture Size", value=1024, step=512)
            
            with gr.Row():
                extract_glb_btn = gr.Button("Extract GLB", interactive=False)
                extract_gs_btn = gr.Button("Extract Gaussian", interactive=False)

        with gr.Column():
            video_output = gr.Video(label="Generated 3D Asset", autoplay=True, loop=True, height=300)
            model_output = LitModel3D(label="Extracted GLB/Gaussian", exposure=10.0, height=300)
            
            with gr.Row():
                download_glb = gr.DownloadButton(label="Download GLB", interactive=False)
                download_gs = gr.DownloadButton(label="Download Gaussian", interactive=False)
    
    # State management
    output_buf = gr.State()

    # Event handlers
    image_prompt.upload(
        preprocess_image,
        inputs=[image_prompt],
        outputs=[image_prompt],
    )

    generate_btn.click(
        get_seed,
        inputs=[randomize_seed, seed],
        outputs=[seed],
    ).then(
        image_to_3d,
        inputs=[image_prompt, seed, ss_guidance_strength, ss_sampling_steps, slat_guidance_strength, slat_sampling_steps],
        outputs=[output_buf, video_output],
    ).then(
        lambda: tuple([gr.Button(interactive=True), gr.Button(interactive=True)]),
        outputs=[extract_glb_btn, extract_gs_btn],
    )

    extract_glb_btn.click(
        extract_glb,
        inputs=[output_buf, mesh_simplify, texture_size],
        outputs=[model_output, download_glb],
    ).then(
        lambda: gr.Button(interactive=True),
        outputs=[download_glb],
    )
    
    extract_gs_btn.click(
        extract_gaussian,
        inputs=[output_buf],
        outputs=[model_output, download_gs],
    ).then(
        lambda: gr.Button(interactive=True),
        outputs=[download_gs],
    )

print("✅ Gradio interface created successfully!")


In [ ]:
# Launch the Gradio interface
print("🚀 Launching Gradio interface...")
demo.launch(
    share=True,  # Creates a public link that can be shared
    server_port=7860,
    show_error=True,
    debug=True
)


In [ ]:
# Simple example usage without Gradio interface
def simple_example():
    # Create a simple test image (you can replace this with your own image)
    try:
        # Try to load example image if available
        image = Image.open("assets/example_image/T.png")
        print("Using example image")
    except:
        # Create a simple colored square as fallback
        image = Image.new('RGBA', (512, 512), (255, 100, 100, 255))
        print("Using fallback test image (red square)")
    
    # Preprocess the image
    processed_image = pipeline.preprocess_image(image)
    
    # Run the pipeline
    print("🎨 Generating 3D asset... (This may take several minutes)")
    outputs = pipeline.run(
        processed_image,
        seed=42,
        formats=["gaussian", "mesh"],
        sparse_structure_sampler_params={
            "steps": 12,
            "cfg_strength": 7.5,
        },
        slat_sampler_params={
            "steps": 12,
            "cfg_strength": 3.0,
        },
    )
    
    # Render videos
    print("🎬 Rendering videos...")
    video_gs = render_utils.render_video(outputs['gaussian'][0])['color']
    video_mesh = render_utils.render_video(outputs['mesh'][0])['normal']
    
    # Save outputs
    imageio.mimsave("sample_gs.mp4", video_gs, fps=30)
    imageio.mimsave("sample_mesh.mp4", video_mesh, fps=30)
    
    # Extract GLB
    print("📦 Extracting GLB...")
    glb = postprocessing_utils.to_glb(
        outputs['gaussian'][0],
        outputs['mesh'][0],
        simplify=0.95,
        texture_size=1024,
    )
    glb.export("sample.glb")
    
    # Save Gaussian PLY
    outputs['gaussian'][0].save_ply("sample.ply")
    
    print("✅ Generation complete!")
    print("📁 Files generated:")
    print("  - sample_gs.mp4: Gaussian video")
    print("  - sample_mesh.mp4: Mesh video") 
    print("  - sample.glb: 3D model file")
    print("  - sample.ply: Gaussian point cloud")
    
    # Clean up GPU memory
    torch.cuda.empty_cache()

# Uncomment the line below to run the simple example
# simple_example()


In [ ]:
# COLAB SPECIFIC CODE - Use this instead of UV commands above
# Uncomment and run this in Colab:

# # Check GPU
# !nvidia-smi

# # Mount Google Drive for persistent storage (optional)
# from google.colab import drive
# drive.mount('/content/drive')

# # Clone repository
# !git clone --recurse-submodules https://github.com/microsoft/TRELLIS.git
# %cd TRELLIS

# # Install PyTorch
# !pip install torch==2.4.0 torchvision==0.19.0 --index-url https://download.pytorch.org/whl/cu118

# # Install dependencies
# !pip install pillow imageio imageio-ffmpeg tqdm easydict opencv-python-headless scipy ninja rembg onnxruntime trimesh open3d xatlas pyvista pymeshfix igraph transformers
# !pip install git+https://github.com/EasternJournalist/utils3d.git@9a4eb15e4021b67b12c460c7057d642626897ec8

# # Install Gradio
# !pip install "pydantic<2.10,>=2.0" gradio==4.44.1 gradio_litmodel3d==0.0.1

print("Colab adaptation code is commented out. Uncomment the lines above when running in Colab.")
